##### DIA MASS SPEC TO PTM-SEA CONVERSION TOOL
```
Revision version: 1.0 - 11/30/22
First version: 1.0 - 11/30/22

Description: This code is used to transform sequence output from DIA MS to the acceptable flanking sequence format used by PTM-SEA. To use, input what is necessary at the end and run the code. Ensure that your input data file has a "ps" column ie. the p value to be input in PTM-SEA.
------------------------------------
Authors: Daphnee Marciniak, Erik Butcher
University of Washington School of Medicine
Department of Pharmacology
1956 NE Pacific St H362, Seattle, WA (98195)
United States of America
Website:
------------------------------------
Note: Please review instructions before use!
------------------------------------
```

In [1]:
import pandas as pd
import re
from Bio import SeqIO

In [6]:
# Insert your data file here. Must have the columns "Protein.id",
# " Modified.Sequence", "Stripped.Sequence", and "ps"
path = r'<base path>'
file_name = r'<input file name>'

# Insert fasta database file here. Should have .fas, .fasta, or some other
# appropriate fasta ending. Make sure it's the same database used to
# generate your DIA data!
database_file = r'<db file fasa>'

# Insert desired output directory. Remember to put .xlsx at the end
output_file_name = r'output xslx file'

In [ ]:
input_path = r'{:s}\{:s}'.format(path, file_name)
output_path = r'{:s}\{:s}'.format(path, output_file_name)
[input_path, output_path]

In [4]:
def start(file_name):

    # Removes non-UniMod:21 (UniMod:21 is phosphorylated STY)

    peptides = pd.read_excel(file_name)
    mod_seq = []
    for i in peptides["Modified.Sequence"]:
        mod_seq.append(re.split(r"[\( | \)]", i))
    for broken_peptide in mod_seq:
        for item in broken_peptide:
            if item == "":
                broken_peptide.remove(item)
    for broken_peptide in mod_seq:
        for item in broken_peptide:
            for n in range(100):
                if n != 21:
                    if item == "UniMod:" + str(n):
                        broken_peptide.remove(item)

    # Format into a pandas dataframe

    protein_id = []
    stripped = []
    ps = []
    for j in peptides["Protein.Group"]:
        protein_id.append(j)
    for k in peptides["Stripped.Sequence"]:
        stripped.append(k)
    for l in peptides["ps"]:
        ps.append(l)
    data = {"Protein.id": protein_id,
            "Modified.Sequence": mod_seq,
            "Stripped.Sequence": stripped,
            "ps": ps}
    df = pd.DataFrame(data, columns=["Protein.id", "Modified.Sequence",
                                     "Stripped.Sequence", "ps"])

    # In case it wasn't done before, removes rows that do not
    # contain UniMod:21

    for ind, row in df.iterrows():
        mod_seq = row["Modified.Sequence"]
        a = 0
        for item in mod_seq:
            z = re.search("UniMod:21", item)
            if z is None:
                a = a
            else:
                a += 1
        if a == 0:
            df = df.drop(index=ind)

    #     print(df)

    return df


def database(database_file, filetype):

    '''Uses the fasta database to return a dictionary containing the protein id
    as keys and sequences as values.
    By default, decoy proteins have format "rev_sp" while actual proteins are
    just "sp," hence skipping rev_sp.'''

    protein_database = {}
    for seq_record in SeqIO.parse(database_file, filetype):
        a = seq_record.seq
        b = re.split(r"[\| | \|]", seq_record.id)
        if b[0] == "rev_sp":
            continue
        protein_database[b[1]] = a
    #     print(protein_database)

    return protein_database


def separate_phosphosites(df):

    '''If there is more than one UniMod:21, makes new rows with copied data
    that contain the modified sequence with only one UniMod:21.
    Also combines amino acid strings if applicable (adjacent amino acid
    strings in Modified.Sequence are either due to loss of UniMod:21 or the
    other UniMods from the first step).
    Remakes dataframe.'''

    new_mod_seq_list = []
    new_stripped_list = []
    new_protein_list = []
    new_ps_list = []

    for ind, row in df.iterrows():
        sequence = row["Stripped.Sequence"]
        protein = row["Protein.id"]
        mod_seq = row["Modified.Sequence"]
        ps = row["ps"]
        n = 0
        for item in mod_seq:
            if item == "UniMod:21":
                n += 1

        # If more than one "UniMod:21," need to split
        if n > 1:
            o = 1
            for i in range(n):
                m = 0
                new_mod_seq = []
                for item in mod_seq:
                    if item != "UniMod:21":

                        # Concatenates adjacent amino acid strings if
                        # applicable
                        if len(new_mod_seq) > 0 and new_mod_seq[
                            len(new_mod_seq) - 1] != "UniMod:21":
                            concat = (new_mod_seq[len(new_mod_seq) - 1]
                                     ) + item
                            new_mod_seq[len(new_mod_seq) - 1] = concat
                        else:
                            new_mod_seq.append(item)

                    if item == "UniMod:21":
                        m += 1
                        if m == o:
                            new_mod_seq.append(item)
                o += 1
                new_mod_seq_list.append(new_mod_seq)
                new_stripped_list.append(sequence)
                new_protein_list.append(protein)
                new_ps_list.append(ps)
        else:
            new_mod_seq = []
            for item in mod_seq:
                new_mod_seq.append(item)

                # Concatenates adjacent amino acid strings if applicable
                if item != "UniMod:21":
                    if new_mod_seq.index(item) != 0 and new_mod_seq[
                        new_mod_seq.index(item) - 1] != "UniMod:21":
                        concat = (new_mod_seq[new_mod_seq.index(item) - 1]
                                 ) + item
                        new_mod_seq[new_mod_seq.index(item) - 1] = concat
                        new_mod_seq.remove(item)

            new_mod_seq_list.append(new_mod_seq)
            new_stripped_list.append(sequence)
            new_protein_list.append(protein)
            new_ps_list.append(ps)

    data1 = {"Protein.id": new_protein_list,
             "Modified.Sequence": new_mod_seq_list,
             "Stripped.Sequence": new_stripped_list,
             "ps": new_ps_list}

    df1 = pd.DataFrame(data1, columns=["Protein.id", "Modified.Sequence",
                                       "Stripped.Sequence", "ps"])
    #     print(df1)

    return df1


def get_flanking(df, database_file, filetype):

    '''Gets the position of the phosphorylated amino acid in Modified.Sequence
    and lowercases it so it can be found again.
    Searches for matching sequence (in all uppercase) in the given database,
    gets position of phosphorylates amino acid in said database and returns
    flanking +/- 7 amino acids plus "-p" and phosphosite letter + site number
    (ie. Y87).
    If +/- 7 goes beyond the applicable range, "_" is given (acceptable in
    PTM-SEA).
    Adds Flanking column to dataframe.
    Adds Phosphosite column to dataframe.'''

    centered_phosphosite_list = []
    phospho_with_position_list = []

    protein_database = database(database_file, filetype)

    for ind, row in df.iterrows():

        mod_seq = row["Modified.Sequence"]
        protein = row["Protein.id"]

        for item in mod_seq:
            if item == mod_seq[0]:
                new_item = ""
                a = len(item)
                i = 0
                for letter in item:
                    i += 1
                    if i == a:
                        new_item = new_item + letter.lower()
                        phospho_letter = letter
                    else:
                        new_item = new_item + letter
            b = item.replace(item, new_item)

        if len(mod_seq) == 2:
            defined_phosphosite = b
        else:
            defined_phosphosite = b + mod_seq[2]

        if protein in protein_database:
            z = re.search(defined_phosphosite.upper(), str(protein_database[
                                                               protein]))
            if z is None:
                centered_phosphosite_list.append("Sequence not matching")
                phospho_with_position_list.append("Sequence not matching")
            else:
                span = z.span(0)
                database_sequence_list = []
                database_sequence = list(str(protein_database[protein]))
                for letter in database_sequence:
                    database_sequence_list.append(letter)
                for letter in defined_phosphosite:
                    if letter.islower():
                        c = defined_phosphosite.index(letter)
                e = database_sequence
                position = span[0] + c
                centered_phosphosite = ""
                phospho_with_position = phospho_letter + str(position)
                phospho_with_position_list.append(phospho_with_position)
                for i in range(position - 7, position + 8):
                    if i > -1 and i < len(e):
                        centered_phosphosite = ''.join([centered_phosphosite,
                                                        e[i]])
                    if i < 0 or i >= len(e):
                        centered_phosphosite = ''.join([centered_phosphosite,
                                                        "_"])
                centered_phosphosite_list.append(centered_phosphosite + "-p")
        else:
            # raise ValueError("Protein not found in database. Are you using "
            #                  "the correct database?")
            centered_phosphosite_list.append("Not in database")
            phospho_with_position_list.append("Not in database")
    df["Flanking"] = centered_phosphosite_list
    df["Phosphosite"] = phospho_with_position_list
    print(df)
    return df


def df_to_csv(df, output_path):
    df_csv = df.to_csv(output_path)

def df_to_excel(df, output_path):
    writer = pd.ExcelWriter(output_path, engine='xlsxwriter')
    df.to_excel(writer, sheet_name="generated_results")
    writer.save()

In [ ]:
df = start(input_path)

In [ ]:
df_phosphites_separate = separate_phosphosites(df)


In [ ]:
df_flanked = get_flanking(df_phosphites_separate, database_file,"fasta")

In [ ]:
df_to_excel(df_flanked, output_path)